In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
from bs4 import BeautifulSoup
import json

## Chrome Navigator

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe', chrome_options=chrome_options)
driver.maximize_window()
driver.get("https://www.linkedin.com/login/fr?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_key']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='session_password']")))

username.clear()
username.send_keys("")
password.clear()
password.send_keys("")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

<ipython-input-2-09db3160241c>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:\Program Files (x86)\chromedriver.exe', chrome_options=chrome_options)


## Scroll Down To Skills Function

In [3]:
def scrollDownToSkills() :
    founded = False
    for scrol in range(100,5000,100):
        while(founded == False) :
            try :

                driver.execute_script(f"window.scrollTo(0,{scrol})")
                time.sleep(1)
                driver.find_element_by_xpath("//button[@class='pv-profile-section__card-action-bar pv-skills-section__additional-skills artdeco-container-card-action-bar artdeco-button artdeco-button--tertiary artdeco-button--3 artdeco-button--fluid artdeco-button--muted']").click()
                founded = True
                break
            except Exception as e:    
                founded = False
                break

## Get Relation

In [4]:
def getRelation():
    try :
        src = driver.page_source
        soup = BeautifulSoup(src, features="html5lib")
        r = soup.findAll(class_='pv-top-card--list pv-top-card--list-bullet display-flex pb1')
        return r[0].text.strip('\n ').split('\xa0')[0]
    except Exception as e: 
        print('Relation not found !')
        return 0

## Get Experience

In [5]:
def getLastExperienceDate():
    try :
        src = driver.page_source
        soup = BeautifulSoup(src, features="html5lib")
        r = soup.findAll(class_='pv-entity__summary-info pv-entity__summary-info--background-section')
        x = soup.findAll(class_="pv-entity__secondary-title t-14 t-black t-normal")
        last_exp = x[0].text.strip('\n ').split('\n')[0]
        y = soup.findAll(class_='pv-entity__date-range t-14 t-black--light t-normal')
        date = y[0].text.strip(' \n ').split('\n')[1].strip(' ')
        return (last_exp, date)
    
    except Exception as e: 
        print('Experience not found')
        return ("No experience", 0)


## Get Formation

In [6]:
def getFormation() :
    src = driver.page_source
    soup = BeautifulSoup(src, features="html5lib")
    formations = soup.findAll(class_='pv-entity__school-name t-16 t-black t-bold')
    return formations[0].text


In [10]:
def mainFunction ():

    linksFile = open("links.txt", 'r', encoding='utf-8')
    listofProfiles = []

    for line in linksFile.readlines():
        try:

            profileDict = {}
            Skills = []
            link = line[:-1]
            driver.get(link)
            time.sleep(5)

            #Profile_Name
            Profile_Name = driver.find_element_by_xpath("//h1[@class='text-heading-xlarge inline t-24 v-align-middle break-words']").text

            #Profile_Tite
            Profiltitle = driver.find_element_by_xpath("//div[@class='text-body-medium break-words']").text

            #Relation
            Relation = getRelation()

            #Pays_Name
            Pays = driver.find_element_by_xpath("//span[@class='text-body-small inline t-black--light break-words']").text        

            scrollDownSkills()
            time.sleep(1)

            #Experience with Date
            last_exp, Date = getLastExperienceDate()

            #Formation
            Formation = getFormation()

            #Skills_Section
            src = driver.page_source
            soup = BeautifulSoup(src, features="html5lib")
            competences = soup.findAll(class_='pv-skill-category-entity__name-text t-16 t-black t-bold')

            for comp in competences:
                Skills.append(comp.text.strip('\n '))

            profileDict["Profile Name"] = Profile_Name
            profileDict["Profile Title"] = Profiltitle
            profileDict["Relation"] = Relation
            profileDict["Pays"] = Pays
            profileDict["Last Experience"] = last_exp
            profileDict["Date"] = Date
            profileDict["Formation"] = Formation
            profileDict["Skills"] = Skills

            listofProfiles.append(profileDict)

        except Exception as e:
            print ("Erreur",e)
            links_e = open('links_error.txt', 'a+')
            links_e.write(link)
            links_e.write('\n')
            continue

    linksFile.close()
    with open('dataset_linkedin.json', 'w+', encoding='utf-8') as json_file:
        json.dump(listofProfiles, json_file)
